In [1]:
from PyQt5.QtGui import*
from PyQt5.QtWidgets import*
from PyQt5.QtCore import*
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import*
from sklearn.model_selection import*
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV

In [2]:
class deneme(QDialog):
    def __init__(self):
        super().__init__()
        veri_egitim=pd.read_csv("https://www.dropbox.com/s/ex9hv7zl22mm5ay/train.csv?dl=1")
        veri_test=pd.read_csv("https://www.dropbox.com/s/hks7tno8d5siuwf/test.csv?dl=1")
        
        veri_egitim.isnull().sum()
        
        print("AGE NIN TUM SETE ORANI: ",(veri_egitim['Age'].isnull().sum()/veri_egitim.shape[0])*100)
        print()
        print("CABININ TUM SETE ORANI: ",(veri_egitim['Cabin'].isnull().sum()/veri_egitim.shape[0])*100)
        print()
        print("EMBARKED IN TUM SETE ORANI: ",(veri_egitim['Embarked'].isnull().sum()/veri_egitim.shape[0])*100)
        print()
        
        print("AGE OZNITELIGININ ORTALAMASI: %.2f"%(veri_egitim['Age'].mean(skipna=True)))
        print()
        print("AGE OZNITELIGININ MEDIANI: %.2f"%(veri_egitim['Age'].median(skipna=True)))
        print()
        print("AGE OZNITELIGINI STANDARD SAPMASI: %.2f"%(veri_egitim['Age'].std(skipna=True)))
        print()
        egitim_veri=veri_egitim.copy()
        egitim_veri['Age'].fillna(veri_egitim['Age'].median(skipna=True),inplace=True)
        egitim_veri.drop('Cabin',axis=1,inplace=True)
        
        egitim_veri['Embarked'].value_counts()
        egitim_veri['Embarked'].fillna(egitim_veri['Embarked'].value_counts().idxmax(),inplace=True)
        
        egitim_veri['TravelAlone']=np.where((egitim_veri['SibSp']+egitim_veri['Parch'])>0,0,1)
        egitim_veri.drop('SibSp',axis=1,inplace=True)
        egitim_veri.drop('Parch',axis=1,inplace=True)
        egitim_veri.isnull().sum()
        training=pd.get_dummies(egitim_veri,columns=['Pclass','Sex','Embarked'])
        training.drop('Name',axis=1,inplace=True)
        training.drop('Ticket',axis=1,inplace=True)
        training.drop('Sex_female',axis=1,inplace=True)
        training.drop("PassengerId",axis=1,inplace=True)
        final_egitim=training.copy()
        final_egitim.isnull().sum()
        
        veri_test.head()
        
        veri_test.drop('Cabin',axis=1,inplace=True)
        veri_test['Age'].fillna(veri_test['Age'].median(skipna=True),inplace=True)
        veri_test['Fare'].fillna(veri_test['Fare'].median(skipna=True),inplace=True)
        veri_test.isnull().sum()
        
        veri_test['TravelAlone']=np.where((veri_test['SibSp']+veri_test['Parch'])>0,0,1)
        veri_test.drop('SibSp',axis=1,inplace=True)
        veri_test.drop('Parch',axis=1,inplace=True)
        
        testing=pd.get_dummies(veri_test,columns=['Pclass','Embarked','Sex'])
        testing.drop('Sex_female',axis=1,inplace=True)
        testing.drop('Name',axis=1,inplace=True)
        testing.drop('Ticket',axis=1,inplace=True)
        testing.drop('PassengerId',axis=1,inplace=True)
        final_test=testing.copy()
        final_test.isnull().sum()
        
        final_egitim['IsMinor']=np.where(final_egitim['Age']<=16,1,0)
        final_test['IsMinor']=np.where(final_test['Age']<=16,1,0)
        
        kolon=['Age','Fare','TravelAlone','Pclass_1','Pclass_2','Embarked_C','Embarked_S','Sex_male','IsMinor']
        x=final_egitim[kolon]
        y=final_egitim['Survived']
        model_lg=LogisticRegression()
        rfe=RFE(model_lg)
        rfe=rfe.fit(x,y)

        print("SECILEN OZNITELIKLER: %s"%(x.columns[rfe.support_]))
        print()
        
        rfecv=RFECV(estimator=model_lg,step=1,cv=10,scoring='accuracy')
        rfecv.fit(x,y)
        
        param_grid={'C':np.arange(1e-05,3,0.1),'penalty':['l1','l2']}
        scoring={'Accuracy':'accuracy'}
        self.lg=GridSearchCV(model_lg,return_train_score=True,param_grid=param_grid,cv=10,refit='Accuracy')
        self.lg.fit(x,y)
        
        print("\nEN IYI PARAMETRELER: %s",self.lg.best_estimator_)
        print()
        print("OPTIMUM HIPERPARAMETRELER: %s"+str(self.lg.best_params_))
        print()
        print("OPTIMUM SKOR: %.2f"%(self.lg.best_score_))
        print()
        
        final_tahmin=self.lg.best_estimator_.predict(x)
        print("EGITIM DOGRULUK ORANI: %.2f"%accuracy_score(final_egitim['Survived'],final_tahmin))
        
        final_test['Survived']=self.lg.best_estimator_.predict(final_test[kolon])
        y_test=final_test['Survived']
        
        survived_test=pd.read_csv("https://www.dropbox.com/s/5jjq3u6ii5zs9pj/gender_submission.csv?dl=1")
        print("TEST DOGRULUK ORANI: %.2f"%accuracy_score(survived_test['Survived'],y_test))
        
        final_test.tail()
        
        self.izgara=QGridLayout()
        self.izgara.addWidget(QLabel("YAS GIRINIZ: "),0,0)
        self.yas=QLineEdit()
        self.izgara.addWidget(self.yas,0,1)
        
        self.izgara.addWidget(QLabel("BILET UCRETI GIRINIZ: "),1,0)
        self.bilet=QLineEdit()
        self.izgara.addWidget(self.bilet,1,1)
        
        self.izgara.addWidget(QLabel("SEYAHAT DURUMU(1-YANLIZ/0-AILE ILE): "),2,0)
        self.TravelAlone=QLineEdit()
        self.izgara.addWidget(self.TravelAlone,2,1)
        
        self.izgara.addWidget(QLabel("BIRINCI SINIF SEYAHAT(1-EVET/0-HAYIR): "),3,0)
        self.sinif_1=QLineEdit()
        self.izgara.addWidget(self.sinif_1,3,1)
        
        self.izgara.addWidget(QLabel("IKINCI SINIF SEYAHAT(1-EVET/0-HAYIR): "),4,0)
        self.sinif_2=QLineEdit()
        self.izgara.addWidget(self.sinif_2,4,1)
        
        self.izgara.addWidget(QLabel("C KAPISI BINIS(1-EVET/0-HAYIR): "),5,0)
        self.binis_c=QLineEdit()
        self.izgara.addWidget(self.binis_c,5,1)
        
        self.izgara.addWidget(QLabel("S KAPISI BINIS(1-EVET/0-HAYIR): "),6,0)
        self.binis_s=QLineEdit()
        self.izgara.addWidget(self.binis_s,6,1)
        
        self.izgara.addWidget(QLabel("CINSIYET GIRINIZ:(1-ERKEK/0-KADIN): "),7,0)
        self.cinsiyet=QLineEdit()
        self.izgara.addWidget(self.cinsiyet,7,1)
        
        self.izgara.addWidget(QLabel("16 YASINDAN KUCUK MU(1-EVET/0-HAYIR): "),8,0)
        self.yas_araligi=QLineEdit()
        self.izgara.addWidget(self.yas_araligi,8,1)
        
        self.buton=QPushButton("LOGISTIC REGRESSION")
        self.izgara.addWidget(self.buton,9,0)
        self.buton.clicked.connect(self.tikla)
        
        self.izgara.addWidget(QLabel("SONUC: "),10,0)
        self.sonuc=QLabel("SONUC BOLUMU...")
        self.izgara.addWidget(self.sonuc,10,1)
        
        
        self.setLayout(self.izgara)
        self.setWindowTitle("TITANIC")
        self.setGeometry(100,100,400,400)
    def tikla(self):
        liste=[]
        liste.append(int(self.yas.text()))
        liste.append(int(self.bilet.text()))
        liste.append(int(self.TravelAlone.text()))
        liste.append(int(self.sinif_1.text()))
        liste.append(int(self.sinif_2.text()))
        liste.append(int(self.binis_c.text()))
        liste.append(int(self.binis_s.text()))
        liste.append(int(self.cinsiyet.text()))
        liste.append(int(self.yas_araligi.text()))
        dene=np.array([liste])
        tahmin_veri=self.lg.best_estimator_.predict(dene)
        if(tahmin_veri==1):
            self.sonuc.setText("<font color='red'><b>BILGILERI GIRILEN KISI KURTULMUSTUR...</b></font>")
        else:
            self.sonuc.setText("<font color='red'><b>BILGILERI GIRILEN KISI MAALESEF KURTULAMAMISTIR...</b></font>")
        
        
        
app=QApplication(sys.argv)
pencere=deneme()
pencere.show()
sys.exit(app.exec_())

AGE NIN TUM SETE ORANI:  19.865319865319865

CABININ TUM SETE ORANI:  77.10437710437711

EMBARKED IN TUM SETE ORANI:  0.22446689113355783

AGE OZNITELIGININ ORTALAMASI: 29.70

AGE OZNITELIGININ MEDIANI: 28.00

AGE OZNITELIGINI STANDARD SAPMASI: 14.53



C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

SECILEN OZNITELIKLER: Index(['Pclass_1', 'Pclass_2', 'Sex_male', 'IsMinor'], dtype='object')



C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

...


EN IYI PARAMETRELER: %s LogisticRegression(C=2.4000100000000004)

OPTIMUM HIPERPARAMETRELER: %s{'C': 2.4000100000000004, 'penalty': 'l2'}

OPTIMUM SKOR: 0.80

EGITIM DOGRULUK ORANI: 0.80
TEST DOGRULUK ORANI: 0.94


ValueError: invalid literal for int() with base 10: ''

C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


SystemExit: 0

C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
